### Conexion con aws

In [4]:
import boto3

# Crear cliente de DynamoDB
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

# Listar tablas
try:
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])
    print("Tablas en DynamoDB:")
    for tabla in tablas:
        print(f"- {tabla}")
except Exception as e:
    print("Error al listar tablas:", str(e))

Tablas en DynamoDB:


### 1 -Crear al menos 3 tablas con dos atributos cada una

In [9]:
# Crear tablas en DynamoDB
try:
    # Tabla 1
    dynamodb_client.create_table(
        TableName='Tabla1',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Apellido', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Apellido', 'AttributeType': 'S'}
            
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    # Tabla 2
    dynamodb_client.create_table(
        TableName='Tabla2',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Formacion', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Formacion', 'AttributeType': 'S'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    # Tabla 3
    dynamodb_client.create_table(
        TableName='Tabla3',
        KeySchema=[
            {'AttributeName': 'Nombre', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'Profesion', 'KeyType': 'RANGE'}  # Sort key
        ],
        AttributeDefinitions=[
            {'AttributeName': 'Nombre', 'AttributeType': 'S'},
            {'AttributeName': 'Profesion', 'AttributeType': 'S'}

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

    print("Tablas creadas exitosamente.")
except Exception as e:
    print("Error al crear tablas:", str(e))

Tablas creadas exitosamente.


### 2 - Crear tres registros

In [20]:
# Crear registros en las tablas
try:
    # Registro en Tabla1
    dynamodb_client.put_item(
        TableName='Tabla1',
        Item={
            'Nombre': {'S': 'Juan'},
            'Apellido': {'S': 'Perez'},
            'Edad': {'N': '17'}
        }
    )
    dynamodb_client.put_item(
        TableName='Tabla1',
        Item={
            'Nombre': {'S': 'Carlos'},
            'Apellido': {'S': 'Gomez'},
            'Edad': {'N': '22'}
        }
    )

    # Registro en Tabla2
    dynamodb_client.put_item(
        TableName='Tabla2',
        Item={
            'Nombre': {'S': 'Ana'},
            'Formacion': {'S': 'Ingenieria'},
            'Edad': {'N': '28'}
        }
    )
    dynamodb_client.put_item(
        TableName='Tabla2',
        Item={
            'Nombre': {'S': 'Pedro'},
            'Formacion': {'S': 'Farmaceutico'},
            'Edad': {'N': '37'}
        }
    )

    # Registro en Tabla3
    dynamodb_client.put_item(
        TableName='Tabla3',
        Item={
            'Nombre': {'S': 'Luis'},
            'Profesion': {'S': 'Medico'},
            'Edad': {'N': '48'}
        }
    )
    dynamodb_client.put_item(
        TableName='Tabla3',
        Item={
            'Nombre': {'S': 'Marcos'},
            'Profesion': {'S': 'Abogado'},
            'Edad': {'N': '57'}
        }
    )

    print("Registros creados exitosamente.")
except Exception as e:
    print("Error al crear registros:", str(e))

Registros creados exitosamente.


### 3 - Obtener un registro de cada tabla

In [29]:
def obtener_un_registro_por_tabla():
    """
    Lista todas las tablas disponibles
    y recupera un único registro de cada tabla (si existe).

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son los primeros registros encontrados.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    resultados = {}

    for tabla in tablas:
        # Obtener un único registro de la tabla
        response = dynamodb_client.scan(TableName=tabla, Limit=1)

        # Extraer el primer ítem si existe
        items = response.get('Items', [])
        if items:
            resultados[tabla] = items[0]
        else:
            resultados[tabla] = None  # Si la tabla está vacía

    return resultados

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    registros = obtener_un_registro_por_tabla()
    for tabla, registro in registros.items():
        print(f"Tabla: {tabla}, Registro: {registro}")

Tabla: Tabla1, Registro: {'Nombre': {'S': 'Carlos'}, 'Edad': {'N': '22'}, 'Apellido': {'S': 'Gomez'}}
Tabla: Tabla2, Registro: None
Tabla: Tabla3, Registro: None


#### 4 - Actualizar un registro de cada tabla

In [30]:
# Crear el cliente de DynamoDB
dynamodb = boto3.resource('dynamodb')

# Función para actualizar el registro de cada tabla
def actualizar_edad(tabla_nombre, nombre_usuario, clave_ordenacion, nueva_edad):
    try:
        # Obtener la referencia de la tabla
        tabla = dynamodb.Table(tabla_nombre)

        # Determinar la clave de ordenación según el nombre de la tabla
        if tabla_nombre == 'Tabla1':
            sort_key = 'Apellido'
        elif tabla_nombre == 'Tabla2':
            sort_key = 'Formacion'
        else:
            sort_key = 'Profesion'

        # Ejecutar la operación de actualización
        response = tabla.update_item(
            Key={
                'Nombre': nombre_usuario,  # Clave de partición
                sort_key: clave_ordenacion  # Clave de ordenación
            },
            UpdateExpression="set Edad = :e",  # Actualizamos el atributo Edad
            ExpressionAttributeValues={
                ':e': nueva_edad  # El nuevo valor para Edad
            },
            ReturnValues="ALL_NEW"  # Devuelve los valores después de la actualización
        )
        
        # Mostrar los resultados de la actualización
        print(f"Actualización en {tabla_nombre}: {response['Attributes']}")
    except Exception as e:
        print(f"Error al actualizar en {tabla_nombre}: {e}")
# Actualizar los registros en las tablas
if __name__ == "__main__":
    # Actualizar la Edad de los registros
    actualizar_edad('Tabla1', 'Juan', 'Perez', 23)  # Cambiar Edad de Juan en Tabla1 a 23
    actualizar_edad('Tabla2', 'Ana', 'Ingenieria', 33)   # Cambiar Edad de Ana en Tabla2 a 33
    actualizar_edad('Tabla3', 'Luis', 'Medico', 64)  # Cambiar Edad de Luis en Tabla3 a 64

KeyboardInterrupt: 

#### 5 - Eliminar un registro de cada tabla

In [22]:
# Función para eliminar un registro de cada tabla
def eliminar_registro(tabla_nombre, nombre_usuario, clave_ordenacion):
    try:
        # Obtener la referencia de la tabla
        tabla = dynamodb.Table(tabla_nombre)

        # Determinar la clave de ordenación según el nombre de la tabla
        if tabla_nombre == 'Tabla1':
            sort_key = 'Apellido'
        elif tabla_nombre == 'Tabla2':
            sort_key = 'Formacion'
        else:
            sort_key = 'Profesion'

        # Ejecutar la operación de eliminación
        response = tabla.delete_item(
            Key={
                'Nombre': nombre_usuario,  # Clave de partición
                sort_key: clave_ordenacion  # Clave de ordenación
            }
        )
        
        # Mostrar los resultados de la eliminación
        print(f"Registro eliminado en {tabla_nombre}: {response}")
    except Exception as e:
        print(f"Error al eliminar en {tabla_nombre}: {e}")

# Eliminar los registros en las tablas
if __name__ == "__main__":
    # Eliminar los registros
    eliminar_registro('Tabla1', 'Juan', 'Perez')  # Eliminar registro de Juan en Tabla1
    eliminar_registro('Tabla2', 'Ana', 'Ingenieria')  # Eliminar registro de Ana en Tabla2
    eliminar_registro('Tabla3', 'Luis', 'Medico')  # Eliminar registro de Luis en Tabla3

Registro eliminado en Tabla1: {'ResponseMetadata': {'RequestId': '4LGRRIL23O1BFDGE1LTP5HSK9FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 01 Feb 2025 12:26:20 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '4LGRRIL23O1BFDGE1LTP5HSK9FVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
Registro eliminado en Tabla2: {'ResponseMetadata': {'RequestId': 'NLOEBML3VAVQQREQIKLQII5LK3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 01 Feb 2025 12:26:20 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'NLOEBML3VAVQQREQIKLQII5LK3VV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
Registro eliminado en Tabla3: {'ResponseMetadata': {'RequestId': 'FN0JMTM7C64MM0OTQB4IKF69RBVV4KQNSO5AEMVJF66Q

#### 6 - Obtener todos los registros de cada tabla

In [29]:
def obtener_todos_los_registros():
    """
    Lista todas las tablas disponibles
    y recupera todos los registros de cada tabla.

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son listas de registros.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    resultados = {}

    for tabla in tablas:
        # Obtener todos los registros de la tabla
        response = dynamodb_client.scan(TableName=tabla)
        items = response.get('Items', [])
        resultados[tabla] = items

    return resultados

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    todos_los_registros = obtener_todos_los_registros()
    for tabla, registros in todos_los_registros.items():
        print(f"Tabla: {tabla}, Registros: {registros}")

Tabla: Tabla1, Registros: [{'Nombre': {'S': 'Carlos'}, 'Edad': {'N': '22'}, 'Apellido': {'S': 'Gomez'}}]
Tabla: Tabla2, Registros: [{'Nombre': {'S': 'Pedro'}, 'Edad': {'N': '37'}, 'Formacion': {'S': 'Farmaceutico'}}]
Tabla: Tabla3, Registros: [{'Nombre': {'S': 'Marcos'}, 'Edad': {'N': '57'}, 'Profesion': {'S': 'Abogado'}}]


#### 7 - Obtener un conjunto de registros de un filtrado de cada tabla

In [30]:
def obtener_registros_filtrados(filtro_expresion, valores_atributos):
    """
    Parámetros:
        - filtro_expresion: Una expresión de filtro para aplicar a la operación de escaneo.
        - valores_atributos: Un diccionario de valores de atributos para la expresión de filtro.

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son listas de registros que cumplen con el filtro.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    resultados = {}

    for tabla in tablas:
        # Obtener registros filtrados de la tabla
        response = dynamodb_client.scan(
            TableName=tabla,
            FilterExpression=filtro_expresion,
            ExpressionAttributeValues=valores_atributos
        )
        items = response.get('Items', [])
        resultados[tabla] = items

    return resultados

# Definir la expresión de filtro y los valores de atributos
filtro_expresion = "Edad > :edad"
valores_atributos = {
    ":edad": {"N": "30"}
}

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    registros_filtrados = obtener_registros_filtrados(filtro_expresion, valores_atributos)
    for tabla, registros in registros_filtrados.items():
        print(f"Tabla: {tabla}, Registros filtrados: {registros}")

Tabla: Tabla1, Registros filtrados: []
Tabla: Tabla2, Registros filtrados: [{'Nombre': {'S': 'Pedro'}, 'Edad': {'N': '37'}, 'Formacion': {'S': 'Farmaceutico'}}]
Tabla: Tabla3, Registros filtrados: [{'Nombre': {'S': 'Marcos'}, 'Edad': {'N': '57'}, 'Profesion': {'S': 'Abogado'}}]


#### 8 - Realizar una eliminación condicional de cada tabla

In [23]:
def eliminar_registro_condicional(tabla_nombre, nombre_usuario, clave_ordenacion, condicion_expresion, valores_atributos):
    try:
        # Obtener la referencia de la tabla
        tabla = dynamodb.Table(tabla_nombre)

        # Determinar la clave de ordenación según el nombre de la tabla
        if tabla_nombre == 'Tabla1':
            sort_key = 'Apellido'
        elif tabla_nombre == 'Tabla2':
            sort_key = 'Formacion'
        else:
            sort_key = 'Profesion'

        # Ejecutar la operación de eliminación condicional
        response = tabla.delete_item(
            Key={
                'Nombre': nombre_usuario,  # Clave de partición
                sort_key: clave_ordenacion  # Clave de ordenación
            },
            ConditionExpression=condicion_expresion,
            ExpressionAttributeValues=valores_atributos
        )
        
        # Mostrar los resultados de la eliminación
        print(f"Registro eliminado condicionalmente en {tabla_nombre}: {response}")
    except Exception as e:
        print(f"Error al eliminar condicionalmente en {tabla_nombre}: {e}")

# Definir la expresión de condición y los valores de atributos
condicion_expresion = "Edad > :edad"
valores_atributos = {
    ":edad": 30
}

# Eliminar los registros condicionalmente en las tablas
if __name__ == "__main__":
    eliminar_registro_condicional('Tabla2', 'Pedro', 'Farmaceutico', condicion_expresion, valores_atributos)  # Eliminar condicionalmente en Tabla2
    eliminar_registro_condicional('Tabla3', 'Marcos', 'Abogado', condicion_expresion, valores_atributos)  # Eliminar condicionalmente en Tabla3

Registro eliminado condicionalmente en Tabla2: {'ResponseMetadata': {'RequestId': 'HHU4M01BEEHR57QL3JD7AR8O0RVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 01 Feb 2025 12:27:53 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'HHU4M01BEEHR57QL3JD7AR8O0RVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}
Registro eliminado condicionalmente en Tabla3: {'ResponseMetadata': {'RequestId': 'OAFT2URTN3E1EPAM81NPK9QEDJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 01 Feb 2025 12:27:53 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'OAFT2URTN3E1EPAM81NPK9QEDJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


#### 9 - Obtener un conjunto de datos a través de varios filtros aplicado en cada tabla

In [25]:
def obtener_registros_con_varios_filtros(filtros_expresiones, valores_atributos):
    """
    Parámetros:
        - filtros_expresiones: Una lista de expresiones de filtro para aplicar a la operación de escaneo.
        - valores_atributos: Un diccionario de valores de atributos para las expresiones de filtro.

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son listas de registros que cumplen con los filtros.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    # Diccionario para almacenar los resultados de cada tabla
    resultados = {}

    for tabla in tablas:
        # Combinar las expresiones de filtro con 'AND'
        filtro_combinado = ' AND '.join(filtros_expresiones)

        # Realizar la operación scan en la tabla actual, aplicando el filtro combinado
        response = dynamodb_client.scan(
            TableName=tabla,
            FilterExpression=filtro_combinado,
            ExpressionAttributeValues=valores_atributos
        )
        # Obtener la lista de items (registros) que cumplen con los filtros
        items = response.get('Items', [])
        resultados[tabla] = items

    return resultados

# Definir las expresiones de filtro y los valores de atributos
filtros_expresiones = ["Edad > :edad", "Apellido = :Apellido"]
valores_atributos = {
    ":edad": {"N": "20"},
    ":Apellido": {"S": "Gomez"}
}

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    registros_filtrados_varios = obtener_registros_con_varios_filtros(filtros_expresiones, valores_atributos)
    for tabla, registros in registros_filtrados_varios.items():
        print(f"Tabla: {tabla}, Registros filtrados: {registros}")

Tabla: Tabla1, Registros filtrados: [{'Nombre': {'S': 'Carlos'}, 'Edad': {'N': '22'}, 'Apellido': {'S': 'Gomez'}}]
Tabla: Tabla2, Registros filtrados: []
Tabla: Tabla3, Registros filtrados: []


#### 10 - Utilizar PartiQL statement en cada tabla


In [32]:
def ejecutar_partiql_en_tablas():
    """
    Ejecuta una declaración PartiQL en cada tabla para seleccionar todos los elementos.

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son listas de registros obtenidos.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    resultados = {}

    for tabla in tablas:
        # Para cada tabla, construimos una consulta PartiQL que selecciona todos los registros.
        # La consulta resultante tendrá el formato: SELECT * FROM "NombreDeLaTabla"
        statement = f"SELECT * FROM \"{tabla}\""
        response = dynamodb_client.execute_statement(Statement=statement)
        items = response.get('Items', [])
        resultados[tabla] = items

    return resultados

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    registros_partiql = ejecutar_partiql_en_tablas()
    for tabla, registros in registros_partiql.items():
        print(f"Tabla: {tabla}, Registros: {registros}")

Tabla: Tabla1, Registros: [{'Nombre': {'S': 'Carlos'}, 'Edad': {'N': '22'}, 'Apellido': {'S': 'Gomez'}}]
Tabla: Tabla2, Registros: []
Tabla: Tabla3, Registros: []


#### 11 - Crear un backup de todas las tablas

In [35]:
def crear_backup_de_todas_las_tablas():
    """
    Crea un backup de todas las tablas disponibles en DynamoDB.

    Retorna:
        - Un diccionario donde las claves son los nombres de las tablas 
          y los valores son los ARNs de los backups creados.
    """
    # Obtener lista de tablas
    response = dynamodb_client.list_tables()
    tablas = response.get('TableNames', [])

    backups = {}

    for tabla in tablas:
        try:
            # Crear un backup para la tabla actual
            response = dynamodb_client.create_backup(
                TableName=tabla,
                BackupName=f"{tabla}_backup"
            )
            backups[tabla] = response['BackupDetails']['BackupArn']
            print(f"Backup creado para la tabla {tabla}: {response['BackupDetails']['BackupArn']}")
        except Exception as e:
            print(f"Error al crear backup para la tabla {tabla}: {e}")

    return backups

# Ejecutar la función y mostrar los resultados
if __name__ == "__main__":
    backups_creados = crear_backup_de_todas_las_tablas()
    for tabla, backup_arn in backups_creados.items():
        print(f"Tabla: {tabla}, Backup ARN: {backup_arn}")

Backup creado para la tabla Tabla1: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla1/backup/01738413370523-517c8115
Backup creado para la tabla Tabla2: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla2/backup/01738413370710-c1e152bc
Backup creado para la tabla Tabla3: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla3/backup/01738413370878-4db403d6
Tabla: Tabla1, Backup ARN: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla1/backup/01738413370523-517c8115
Tabla: Tabla2, Backup ARN: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla2/backup/01738413370710-c1e152bc
Tabla: Tabla3, Backup ARN: arn:aws:dynamodb:us-east-1:959174264772:table/Tabla3/backup/01738413370878-4db403d6
